# Fine Tuning em Hiperparâmetros

Os hiperparâmetros são valores que são definidos pelo(s) criador(es) do modelo para que ele funcione da forma esperada. Não há um valor pré-definido que irá funcionar perfeitamente para um modelo qualquer, então é preciso ajustar um valor que obterá uma performance melhor para o modelo. Esse processo é conhecido como fine tuning.

O processo depende que alguns valores sejam testados no modelo e que a performance do modelo seja avaliada constantemente até que um valor razoável seja encontrado para o hiperparâmetro.

Para esse modelo, os hiperparâmetros que passarão pelo processo de fine tuning são:

- número de épocas (epoch)
- taxa de aprendizado (learning rate)
- taxa de decaimento (weight decay)

## Fine tuning para 'Epoch'

O processo de fine tuning para o hiperparâmetro de epoch será realizado de forma isolada. Isso significa que valores para os outros hiperparâmetros permanecerão fixos, enquanto o modelo é treinado para um número 'n' de epochs. Por meio dos passos de validação, durante o treinamento do modelo, é possível verificar a performance do modelo enquanto ele é treinado ao longo das épocas.

Configuração:

```
    'BATCH_SIZE': 4,
    'MAX_NUMBER_TOKENS': 512,
    'NUMBER_OF_BRANCHES': 13,
    'EPOCHS': 10,
    'LEARNING_RATE': 2e-5,
    'WEIGHT_DECAY': 0.01,
    'WARM_UP_PROPORTION': 0.1
```

## Fine tuning para 'Taxa de Aprendizado' e 'Taxa de Decaimento'

Para definir os valores dos hiperparâmetros 'taxa de aprendizado' e 'taxa de aquecimento', alguns valores foram escolhidos, de acordo com as sugestões [desse paper](https://arxiv.org/pdf/1905.05583.pdf). Dessa forma, todas as combinações de cada um dos valores que cada hiperparâmetro pode assumir será testado:

- Taxa de aprendizado: 2.5e-5, 2e-5
- Taxa de decaimento: 0.001, 0.01
- Warm up proportion: 0.1, 0.3


## Inicialização e definiçao de constantes

Como uma etapa inicial, toda a inicialização do notebook será concentrada no início desse documento. Os conteúdos contidos aqui são:

1. Instalação de bibliotecas externas
2. Importação de biblioteca
3. Definição de valores constantes que podem ter seu uso replicado ao longo do notebook
4. Inicialização do sistema de arquivos integrado ao Google Drive

Ao fim dessa seção, os hiperparâmetros são definidos.

In [ ]:
# Installation of 3rd party libraries

!pip install transformers
!pip install --upgrade pytorch-lightning

     |████████████████████████████████| 2.8 MB 12.0 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 895 kB 45.6 MB/s 
     |████████████████████████████████| 3.3 MB 32.4 MB/s 
     |████████████████████████████████| 636 kB 35.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 922 kB 12.8 MB/s 
     |████████████████████████████████| 282 kB 53.4 MB/s 
     |████████████████████████████████| 829 kB 42.3 MB/s 
     |████████████████████████████████| 119 kB 49.0 MB/s 
     |████████████████████████████████| 1.3 MB 55.4 MB/s 
     |████████████████████████████████| 142 kB 54.2 MB/s 
     |████████████████████████████████| 294 kB 38.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=f2c3c8b92385527028124e32109104a2782fadc9f92d094c2591f48ab29d107b
  Sto

In [ ]:
# Imports

from google.colab import drive
import pandas as pd
import numpy as np
from transformers import BertTokenizerFast as BertTokenizer, BertForSequenceClassification, AdamW, get_cosine_schedule_with_warmup, Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import re
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Constants

CONSTANTS = {
    'TRAINING_DATASET': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Train_Dataset.csv',
    'VALIDATION_DATASET': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Validation_Dataset.csv',
    'BERT_MODEL_NAME': 'neuralmind/bert-large-portuguese-cased',
    'SEED': 13
}

# Hyperparameters

HYPERPARAMETERS = {
    'BATCH_SIZE': 2,
    'MAX_NUMBER_TOKENS': 512,
    'NUMBER_OF_BRANCHES': 13,
    'EPOCHS': 3,
    'LEARNING_RATE': 2.5e-5,
    'WEIGHT_DECAY': 0.001,
    'WARM_UP_PROPORTION': 0.1
}

In [ ]:
# Mounting Google Drive

drive.mount('/content/drive', force_remount=True)

## Verificar disponibilidade da GPU

O próximo passo seria verificar se a GPU oferida pela Google gratuitamente como ambiente de execução do notebook está funcionando corretamente. A GPU oferece uma performance computacional maior em relação a calculos sendo executados pela CPU.

In [ ]:
torch.cuda.empty_cache()

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

### Reproducibilidade

Para fins de reproducibilidade, definimos uma semente para o pytorch lightning.

In [ ]:
seed_everything(CONSTANTS['SEED'])

## Carregar os dados

Com os arquivos em mãos, é possível carregá-los para que os dados contidos possam ser utilizados para a criação do modelo.

Nessa etapa, os dados são carregados a partir de arquivos .csv contendo as informações dos acórdãos. Esses arquivos .csv já foram pre-processados no notebook Data_Preprocessing.ipynb, que se encontra na pasta Projeto do Google Drive. No pre-processamento as classificações dos ramos do direito de cada acórdão são mapeadas para valores numéricos que o BERT consiga entender. Esse mapeamento segue o seguinte conjunto de chaves e valores:
- Direito Penal (Direito Processual Penal) &rarr; 0
- Direito Administrativo (Licitações, Contratos Administrativos, Servidores, Desapropriação, Tribunal de Contas, Improbidade, etc.) &rarr; 1
- Direito Tributário/Direito Financeiro &rarr; 2
- Direito Civil (Direito Comercial/Direito de Família) &rarr; 3
- Direito Previdenciário &rarr; 4
- Direito do Trabalho &rarr; 5
- Direito Processual Civil &rarr; 6
- Direito Eleitoral &rarr; 7
- Direito do Consumidor &rarr; 8
- Direito Internacional (Público ou Privado) &rarr; 9
- Direito Militar &rarr; 10
- Direito Econômico (Direito concorrencial e Agências Reguladoras Setoriais, Intervenção no Domínio Econômico) &rarr; 11
- Direito Ambiental &rarr; 12

Há dois conjunto de dados a serem carregados: treinamento e validação.

In [ ]:
# Read the training dataset from .csv file
documents = pd.read_csv(CONSTANTS['TRAINING_DATASET'])
documents

In [ ]:
# Read the validation dataset from .csv file
documents_val = pd.read_csv(CONSTANTS['VALIDATION_DATASET'])
documents_val

## Definição dos conjunto de dados de acordo com o tamanho da ementa

O Bert tem uma limitação de lidar apenas com textos de, no máximo, 512 tokens. Portanto, uma forma de contornar a situação seria utilizar apenas o início da ementa até atingir essa capacidade máxima que o Bert oferece.

Uma imagem que explica o método que será utilizado:

![long-sequences-bert](https://drive.google.com/uc?export=view&id=1I-VK8Zy_SZurOl41es8gNSRYtsGZsSfq)

No caso, quando um acórdão possui uma ementa grande, apenas os 512 tokens iniciais serão utilizados.

Para uma abordagem como essa, não é necessário ter porções de implementação para tratar os acórdãos maiores. O BertTokenizer irá lidar com essas ementas maiores truncando o conteúdo para reduzir o tamanho de acordo com sua limitação.

## Preparação para o treinamento do modelo

Depois de ter os dados organizados, a melhor forma de treinar o modelo é antecipar uma preparação dos dados. Dessa forma, geralmente cria-se um Dataset para que o modelo possa consumir os dados facilmente.

O Dataset auxilia a modularizar o código utilizado para treinar o modelo. Dessa forma, as rotinas para manter uma coleção de dados utilizada para o modelo podem ser isoladas no Dataset. O Dataset basicamente compreende amostras de dados com seus respectivos rótulos (saída do modelo).

In [ ]:
class LawDocumentDataset(Dataset):
  def __init__(self, dataframe: pd.DataFrame, tokenizer: BertTokenizer, max_token_length: int=512):
    self.dataframe = dataframe
    self.tokenizer = tokenizer
    self.max_token_length = max_token_length

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, index: int):
    row = self.dataframe.iloc[index]
    summary_document = row.ementa
    law_branch_id = row.ramo

    encoding = self.tokenizer.encode_plus(
      summary_document,
      add_special_tokens=True,          # Add `[CLS]` and `[SEP]`
      max_length=self.max_token_length,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,                  # Truncate encoding to the max length
      return_attention_mask=True,       # Return attention mask
      return_tensors="pt"               # Return PyTorch tensor
    )

    labels = np.eye(HYPERPARAMETERS['NUMBER_OF_BRANCHES'])[law_branch_id]  # Return a list with zeros, except for index law_branch_id that assumes one

    return dict(
        summary_document=summary_document,
        input_ids=encoding["input_ids"].flatten(),
        attention_mask=encoding["attention_mask"].flatten(),
        labels=torch.FloatTensor(labels)
    )

In [ ]:
tokenizer = BertTokenizer.from_pretrained(CONSTANTS['BERT_MODEL_NAME'])
tokenizer

In [ ]:
train_dataset = LawDocumentDataset(documents, tokenizer, HYPERPARAMETERS['MAX_NUMBER_TOKENS'])
validation_dataset = LawDocumentDataset(documents_val, tokenizer, HYPERPARAMETERS['MAX_NUMBER_TOKENS'])

## Criando o modelo

Depois de ter toda a preparação dos dados, o modelo pode então começar a ser treinado. Para isso, utilizar LightiningModule pode auxiliar durante o processo. O código normalmente utilizado para o treinamento de uma rede neural usando Pytorch pode ser compactado por meio do LightningModule. LightningModule permite que o treinamento do modelo esteja disposto de uma forma organizada no código, também prevenindo que chamadas utilizando `.cuda()` ou `.to()` sejam realizadas. A própria classe se responsabiliza para controlar quais tensores devem abrigar cálculos dentro da GPU.

Basicamente, o processo utilizado pelo LightningModule para o treinamento do modelo é:

```
for epoch in range(num_epochs):
    # Training phase
    for batch in train_loader:
        for each entry in batch, run forward
        run training_step
        calculate loss & metrics
    
    # Validation phase
    for batch in validation_loader:
        run validation_step
        calculate loss & metrics
    
    # Test step
    for batch in test_loader:
        run test_step
        calculate loss & metrics
```

In [ ]:
class LawDocumentClassifier(pl.LightningModule):
    
    def __init__(self, number_classes: int, steps_per_epoch: int=None, epochs: int=None, learning_rate: float=2e-5, weight_decay: float=0.01, warm_up_proportion: float=0.1):
        super().__init__()
        
        self.model = BertForSequenceClassification.from_pretrained(
            "neuralmind/bert-large-portuguese-cased",
            num_labels=number_classes,                      # The number of output labels--2 for binary classification
            output_attentions=False,                        # Returns attention weights
            output_hidden_states=False                      # Returns all hidden states
        )
        self.steps_per_epoch = steps_per_epoch
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.warm_up_proportion = warm_up_proportion
        self.weight_decay = weight_decay
        
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids,
                            attention_mask=attention_mask,
                            labels=labels,
                            return_dict=True)
        
        return output.loss, output.logits
        
    def training_step(self, batch, batch_index):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        
        loss, outputs = self(input_ids, attention_mask, labels)
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def compute_metrics(self, eval_pred):
        logits, labels = eval_pred
        
        classification_predictions = self.convert_to_classification_labels(logits)
        classification_labels = self.convert_to_classification_labels(labels)

        metrics = {
            "validation_accuracy": accuracy_score(classification_labels, classification_predictions),
            "validation_precision": precision_score(classification_labels, classification_predictions, average='weighted'),
            "validation_recall": recall_score(classification_labels, classification_predictions, average='weighted'),
            "validation_f1": f1_score(classification_labels, classification_predictions, average='weighted'),
        }

        return metrics
            
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        warmup_steps = self.steps_per_epoch * self.warm_up_proportion
        total_steps = self.steps_per_epoch * self.epochs - warmup_steps

        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        
        return (optimizer, scheduler)

    def convert_to_classification_labels(self, classifications):
        formatted_classifications = []

        for classification in classifications:
            formatted_classifications.append(np.argmax(classification).flatten())

        return formatted_classifications

In [ ]:
model = LawDocumentClassifier(
    HYPERPARAMETERS['NUMBER_OF_BRANCHES'],
    steps_per_epoch=len(documents) // HYPERPARAMETERS['BATCH_SIZE'],
    epochs=HYPERPARAMETERS['EPOCHS'],
    learning_rate=HYPERPARAMETERS['LEARNING_RATE'],
    weight_decay=HYPERPARAMETERS['WEIGHT_DECAY'],
    warm_up_proportion=HYPERPARAMETERS['WARM_UP_PROPORTION']
)

## Treinando o modelo

Basicamente, o processo utilizado pelo LightningModule para o treinamento do modelo é:

```
for epoch in range(num_epochs):
    # Training phase
    for batch in train_loader:
        for each entry in batch, run forward
        run training_step
        calculate loss & metrics
    
    # Validation phase
    for batch in validation_loader:
        run validation_step
        calculate loss & metrics
    
    # Test step
    for batch in test_loader:
        run test_step
        calculate loss & metrics
```

O processo para treinamento do modelo será executado pela API de [Trainer](https://huggingface.co/transformers/main_classes/trainer.html) do HuggingFace. Nesse sentido, alguns parâmetros são definidos durante a criação do objeto `Trainer` para a criação da estratégia de treinamento.

In [ ]:
training_args = TrainingArguments(
    "/content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/trainer_output",
    num_train_epochs=HYPERPARAMETERS['EPOCHS'],
    evaluation_strategy='epoch',
    per_device_train_batch_size=HYPERPARAMETERS['BATCH_SIZE'],
    logging_steps=30
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=model.compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=model.configure_optimizers()
)
trainer.train()

## Análise de métricas

Após a realização do treinamento do modelo, o ideal é extrair as métricas que foram coletadas durante os processos de treinamento e validação em cada uma das épocas, para que então os melhores valores de hiperparâmetros sejam extraídos.

A API `Trainer` de HuggingFace possui a capacidade de calcular automaticamente as métricas configuradas e coletar os valores durante as épocas. Esses valores serão extraídos para análises, como parte do processo de fine tuning dos hiperparâmetros.

In [ ]:
validation_metrics = trainer.evaluate()

In [ ]:
validation_metrics

In [ ]:
# torch.save(model.state_dict(), '/content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/truncated-2e5-0.01-0.3.bin')

- https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
- https://arxiv.org/pdf/1711.05101.pdf
- https://arxiv.org/pdf/1905.05583.pdf
- https://arxiv.org/pdf/1810.04805.pdf
- https://www.fast.ai/2018/07/02/adam-weight-decay/
- Weight decay values: https://openreview.net/pdf?id=Syx4wnEtvH

Cross-entropy Loss
Optimizer = AdamW (valores podem ser retirados [daqui](https://arxiv.org/pdf/1905.05583.pdf))
Scheduler = Cosine, provavelmente sem reset (Appendix C do [paper](https://arxiv.org/pdf/1711.05101.pdf))